In [1]:
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.orm import Session
from sqlalchemy import MetaData
from sqlalchemy import ForeignKey
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import insert
from sqlalchemy import select
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer,Float, String, MetaData
from sqlalchemy.orm import Session


In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [7]:
# Load data using Pandas (replace 'your_data.csv' with your dataset file)
data = pd.read_csv('C://Study//CSE 4000//New folder//Thesis//dataset//New folder//weather_data_sf_2004_2023.csv')

# Get the column names and data types from the CSV
column_names = data.columns
column_data_types = {col: data[col].dtype for col in column_names}

# Create a new MetaData object and a users table based on the column names and data types
metadata = MetaData()

users = Table(
    'users',
    metadata,
    Column('id', Integer, primary_key=True),
    *[Column(name, String) if dtype == 'object' else
      Column(name, Integer) if dtype == 'int64' else
      Column(name, Float) for name, dtype in column_data_types.items()]
)

# Create the table in the database
metadata.create_all(engine)

# Start a session
session = Session(engine)

# Loop through the data and insert into the table, skipping rows with null values
for _, row in data.iterrows():
    user = {name: row[name] for name in column_names if not pd.isna(row[name])}
    session.execute(users.insert().values(user))
# SELECT the first few rows with column names
query = users.select().limit(5)  # You can adjust the limit as needed
result = session.execute(query)

# Get the column names and the data
column_names = result.keys()
data = result.fetchall()

# Print the column names
print(column_names)

# Print the first few rows of data
for row in data:
    print(row)

# Close the session
session.close()

# Commit changes
session.commit()

2023-10-20 04:36:36,156 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-20 04:36:36,157 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-10-20 04:36:36,158 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-20 04:36:36,160 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-10-20 04:36:36,162 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-20 04:36:36,164 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	"Unnamed: 0" INTEGER, 
	expire_time_gmt INTEGER, 
	valid_time_gmt INTEGER, 
	"temp" FLOAT, 
	wx_phrase VARCHAR, 
	heat_index FLOAT, 
	rh FLOAT, 
	pressure FLOAT, 
	vis FLOAT, 
	wc FLOAT, 
	wdir FLOAT, 
	wspd FLOAT, 
	max_temp FLOAT, 
	min_temp FLOAT, 
	expire_date_time VARCHAR, 
	valid_date_time VARCHAR, 
	PRIMARY KEY (id)
)


2023-10-20 04:36:36,165 INFO sqlalchemy.engine.Engine [no key 0.00114s] ()
2023-10-20 04:36:36,167 INFO sqlalchemy.engine.Engine COMMIT
2023-10-20 04:36:36,170 INFO sqlalchemy.engine.Engine BEGI

2023-10-20 04:36:36,182 INFO sqlalchemy.engine.Engine INSERT INTO users ("Unnamed: 0", expire_time_gmt, valid_time_gmt, "temp", wx_phrase, heat_index, rh, pressure, vis, wc, wdir, wspd, expire_date_time, valid_date_time) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2023-10-20 04:36:36,184 INFO sqlalchemy.engine.Engine [cached since 0.006489s ago] (2, 1072958160, 1072950960, 11.0, 'Cloudy', 11.0, 89.0, 1012.84, 8.0, 11.0, 120.0, 20.0, '01/01/2004 18:00:00', '01/01/2004 16:00:00')
2023-10-20 04:36:36,187 INFO sqlalchemy.engine.Engine INSERT INTO users ("Unnamed: 0", expire_time_gmt, valid_time_gmt, "temp", wx_phrase, heat_index, rh, pressure, vis, wc, wdir, wspd, expire_date_time, valid_date_time) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2023-10-20 04:36:36,188 INFO sqlalchemy.engine.Engine [cached since 0.009913s ago] (3, 1072958640, 1072951440, 11.0, 'Cloudy', 11.0, 88.0, 1012.85, 6.0, 11.0, 120.0, 24.0, '01/01/2004 18:00:00', '01/01/2004 16:00:00')
2023-10-20 04:36:36,19